In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# pair_classes = pd.read_csv("../data/Banco de Súmulas - Pair Classes.csv")
# single_classes = pd.read_csv("../data/Banco de Súmulas - Single Classes.csv")
# df = pd.concat([pair_classes, single_classes], ignore_index=True)

# df.to_excel("../data/All Classes.xlsx", index=False)

In [3]:
df = pd.read_excel('../data/All Classes.xlsx')
campeonatos = pd.read_csv('../data/Banco de Súmulas - Campeonatos.csv')

# **> Ajuste do ID da coluna `ID Campeonato`**

In [4]:
def fixIDCampeonato(df, campeonatos):
    """
    Função que ajusta o ID do campeonato de acordo com o nome do campeonato.
    Recebe como parâmetro o dataframe e o dataframe de campeonatos.
    """
    # turn Campeonatos into a dictionary
    campeonatos_dict = {}
    for index, row in campeonatos.iterrows():
        campeonatos_dict[row['Nome Competição']] = row['ID Competição']
    
    # fix ID Campeonato
    for index, row in df.iterrows():
        campeonato = row['Nome Competição']
        try:
            df.at[index, 'ID Competição'] = campeonatos_dict[campeonato]
        except:
            pass
    
    return df

In [5]:
df = fixIDCampeonato(df, campeonatos)

# **> Ajuste do formato da coluna `Pontuação Regata`**

In [6]:
df.head()

,ID Resultado,ID Competidor,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Descarte,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição
0,NaN,NaN,"WILLIAM JONES, EVAN DEPAUL",17.0,49er,11.0,1.0,GERAL,1,UFD,44.0,33.0,World Championship 2017
1,NaN,NaN,"DOMINIK BUKSAK, SZYMON WIERZBICKI",7.0,49er,3.0,NaN,GERAL,1,NaN,89.0,66.0,Semaine Olympique Francaise De Voile 2022
2,NaN,NaN,"DOMINIK BUKSAK, SZYMON WIERZBICKI",7.0,49er,1.0,NaN,GERAL,1,NaN,89.0,66.0,Semaine Olympique Francaise De Voile 2022
3,NaN,NaN,"DOMINIK BUKSAK, SZYMON WIERZBICKI",7.0,49er,1.0,NaN,GERAL,1,NaN,89.0,66.0,Semaine Olympique Francaise De Voile 2022
4,NaN,NaN,"DOMINIK BUKSAK, SZYMON WIERZBICKI",7.0,49er,3.0,NaN,GERAL,1,NaN,89.0,66.0,Semaine Olympique Francaise De Voile 2022


**Casos de exemplo para tratamento:**

[x] 1.0

[x] 2.8

[x] 1

[x] -1

[x] (9.0)

[x] (9)

[x] (32.0 DNC)

[x] DSQ(69)

[x] DNF (32)

[x] (59.0 I BFD)

[x] (54.0 FL-SCP)

[x] DNF

[x] dnf lower case

[x] DNF 32

[x] RDG24

[x] RDG11.75

[x] 24.0 M-SCP

[x] 30.0 ARB

[x] 5.8 RDG F

[x] 4*BFD

[x] *18

In [7]:
def remove_parenthesis_or_asterisk(x):
    """
    Função que remove os parênteses e asteriscos de uma string.
    """
    if x.__contains__('('):
        return x.replace('(', '').replace(')', '')
    elif x.__contains__('*'):
        return x.replace('*', '')

def is_number(x):
    """
    Função que verifica se uma string é um número.
    """
    try:
        float(x)
        return True
    except:
        return False
    
def update(column, current_value, new_value, df, position):
    """
    Função que atualiza o valor de uma célula de um dataframe.
    Recebe como parâmetro o nome da coluna, o valor atual, o novo valor, o dataframe e a posição.
    Se o valor atual for None, o novo valor é atribuído à célula. Caso contrário, o valor atual é mantido.
    """
    if current_value == None:
        df.at[position, column] = new_value
    else:
        df.at[position, column] = current_value

def fixPontuacaoRegata(df):
    """
    Função que ajusta o formato da pontuação da regata considerando todos os casos específicos.
    Recebe como parâmetro o dataframe.
    """
    # define a lista de punicoes
    PUNICOES = ['SPI', 'ARB', 'DCT', 'TLE', 'DNE', 'RET', 'DNF', 'DNC', 
                'DPI', 'DP', 'SCP', 'BFD', 'DNS', 'UFD', 'DSQ', 'STP', 
                'OCS', 'RAF', 'RDG', 'ret', 'ufd', 'bfd', 'PRP', 
                'NSC', 'dsq', 'dns', 'dnf']
    
    df = df.dropna(subset=['Pontuação Regata'])
    df['Descarte'] = df['Descarte'].replace(np.NaN, None)
    df['Punição'] = df['Punição'].replace(np.NaN, None)
    
    for i, row in df.iterrows():
        pontos = row['Pontuação Regata']
        descarte = row['Descarte']
        punicao = row['Punição']
        
        if pontos.startswith('-'):
            """
            Caso do bug do cntrl+c e cntrl+v, em que o número entre parênteses se tornou um número negativo
            """
            pontos = pontos[1:]
            
        if pontos.__contains__(','):
            """
            Caso em que a pontuação da regata é um número decimal, i.e., contém vírgula.
            Nesse caso, devemos substituir a vírgula por ponto.
            """
            df.at[i, 'Pontuação Regata'] = pontos.replace(',', '.')
            
        if is_number(pontos):
            """
            Se pontos for um número, i.e., não tiver qualquer outro texto associado à string.
            Na coluna de Descarte, devemos adicionar 0 se o valor atual for np.nan.
            Na coluna de Punição, devemos manter o valor atual.
            """
            pontos = float(pontos)
            # change in the dataset itself
            df.at[i, 'Pontuação Regata'] = pontos
            update(column='Descarte', current_value=descarte, new_value=0, df=df, position=i)
            #update(column='Punição', current_value=punicao, new_value=punicao, df=df, position=i)

        elif pontos.__contains__('(') or pontos.__contains__('*'):
            """
            Se pontos for uma string que contém um parênteses, pode ser um dos casos abaixo:
            (9.0), (32.0 DNC), DNF (32), DSQ(69).
            Nesse caso, se a remoção dos parênteses resultar em um número, devemos adicionar esse número
            na coluna de pontos. Se não, teremos que passar por mais alguns tratamentos de string.
            """
            pontos = remove_parenthesis_or_asterisk(pontos)
            
            if is_number(pontos):
                """
                Caso em que a remoção dos parênteses resulta apenas em um número.
                Nesse caso, houve descarte mas a indicação da punição não está na string ou não existe.
                Adicionamos 1 na coluna de Descarte, mas mantemos o valor atual na coluna de Punição.
                """
                pontos = float(pontos)
                df.at[i, 'Pontuação Regata'] = pontos
                update(column='Descarte', current_value=descarte, new_value=1, df=df, position=i)
                #update(column='Punição', current_value=punicao, new_value=punicao, df=df, position=i)

            else:
                """
                Casos em que a remoção dos parênteses não resulta em um número puro.
                """
                for p in PUNICOES:
                    if pontos.__contains__(p):
                        """
                        Caso em que a string contém uma punição, i.e., DNC, DNS, etc.
                        Nesse caso, removemos a punição da string, transformamos o valor em float e adicionamos
                        a pontuação na coluna de pontos. Adicionamos 1 na coluna de Descarte e adicionamos a punição
                        na coluna de Punição.
                        """
                        pontos = pontos.replace(p, '')
                        try:
                            pontos = float(pontos)
                            df.at[i, 'Pontuação Regata'] = pontos
                        except:
                            df.at[i, 'Pontuação Regata'] = 'OP' # only punishment
                        update(column='Descarte', current_value=descarte, new_value=1, df=df, position=i)
                        update(column='Punição', current_value=punicao, new_value=p.upper(), df=df, position=i)
                        break
            
        else:
            """
            Casos em que pontos não é um número e não possui parênteses (não há descarte), 
            mas há uma punição na string.
            """
            for p in PUNICOES:
                if pontos.__contains__(p):
                    """
                    Caso em que a string contém uma punição, i.e., DNC, DNS, etc.
                    Nesse caso, removemos a punição da string, transformamos o valor em float e adicionamos
                    a pontuação na coluna de pontos. Adicionamos 0 na coluna de Descarte e adicionamos a punição
                    na coluna de Punição.
                    """
                    pontos = pontos.replace(p, '')
                    try:
                        pontos = float(pontos)
                        df.at[i, 'Pontuação Regata'] = pontos
                    except:
                        df.at[i, 'Pontuação Regata'] = 'OP' # only pushment, 
                    update(column='Descarte', current_value=descarte, new_value=0, df=df, position=i)
                    update(column='Punição', current_value=punicao, new_value=p.upper(), df=df, position=i)
                    break
    
    return df

In [8]:
def fixPontuacaoTotal(df):
    """
    Troca vírgulas por pontos na coluna de Pontuação Total.
    """
    for index, row in df.iterrows():
        pontos = row['Pontuação Total']
        pontos = str(pontos)
        
        if pontos.__contains__(','):
            pontos = pontos.replace(',', '.')
            pontos = float(pontos)
            df.at[index, 'Pontuação Total'] = pontos
    
    return df

In [20]:
df_ajustado = fixPontuacaoRegata(df)

In [21]:
df_ajustado = fixPontuacaoTotal(df_ajustado)

In [22]:
unique = df_ajustado['Nome Competidor'].unique()

#'ID Competidor'
for i, name in enumerate(unique):
    df_ajustado.loc[df_ajustado['Nome Competidor'] == name, 'ID Competidor'] = i+1

In [23]:
df_ajustado.shape

(136618, 13)

In [24]:
df_ajustado.reset_index(inplace=True, drop=True)
df_ajustado.reset_index(inplace=True, drop=False)
df_ajustado["ID Resultado"] = df_ajustado.index + 1
df_ajustado.drop(columns=['index'], inplace=True)

df_ajustado

In [35]:
writer = pd.ExcelWriter('../data/final_data.xlsx')
df_ajustado.to_excel(writer,'Sumulas')

atletas = df_ajustado[["ID Competidor", "Nome Competidor"]].drop_duplicates()
atletas.to_excel(writer,'Atletas')

competicoes = df_ajustado[["ID Competição", "Nome Competição"]].drop_duplicates()
competicoes.to_excel(writer,'Campeonatos')

writer.close()

## **> Obtenção do dataset final**

In [28]:
# # colunas que vão entrar no dsataframe final
# columns_to_model = ['Nome Competidor', 'Classe Vela', 'Flotilha', 'Posição Geral', 'Nett', 'Nome Competição']
# df_to_model = df[columns_to_model]

# # dropa as repetições regata a regata (mantém a primeira)
# df_to_model.drop_duplicates(keep='first', inplace=True)

# df_to_model.head()